In [1]:
!pip install pytorch-lightning==1.0.8
!pip install torch==1.7.0 torchvision==0.8.1 torchtext==0.8.0
!pip install omegaconf==2.0.0 einops>=0.3.0 transformers
!pip install test-tube
!pip install sklearn

     |████████████████████████████████| 561 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 33.7 MB/s 
     |████████████████████████████████| 829 kB 32.3 MB/s 
     |████████████████████████████████| 134 kB 19.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=48836b3b0d01464a04382833a918bfc583f7e42580be4aa4ecaf0577bf8aaf47
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
     |████████████████████████████████| 776.7 MB 5.0 kB/s 
     |████████████████████████████████| 12.7 MB 23.3 MB/s 
     |████████████████████████████████| 6.9 MB 31.7 MB/s 
  At

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os
import glob
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps

In [4]:
def resize_with_padding(img, required_size):
  img.thumbnail((required_size, required_size))
  delta_w = required_size - img.size[0]
  delta_h = required_size - img.size[1]
  pad_w = delta_w // 2
  pad_h = delta_h // 2
  padding = (pad_w, pad_h, delta_w - pad_w, delta_h - pad_h)
  return ImageOps.expand(img, padding, fill=(255,255,255))

images_path = "/content/drive/MyDrive/4_ING_INFORMATICA/tfg/datasets/negev/articles_molecules/"
filenames = glob.glob(images_path+"*.png")

os.makedirs(images_path+"preprocess/", exist_ok=True)

required_size = 192
for filename in filenames:
  img = Image.open(filename)
  width, height = img.size
  img = resize_with_padding(img, required_size)
  
  img.convert("RGB").save(images_path+"preprocess/"+filename.split("/")[-1].replace("png","jpg"))

In [5]:
preprop_images_path = images_path+"preprocess/"
filenames = glob.glob(preprop_images_path+"*.jpg")
train, test = train_test_split(filenames, test_size=0.25, random_state=42)

with open(preprop_images_path+"xx_train.txt", "w") as f:
  for filename in train[:-1]:
    f.write(filename+"\n")
  f.write(train[-1])

with open(preprop_images_path+"xx_test.txt", "w") as f:
  for filename in test[:-1]:
    f.write(filename+"\n")
  f.write(test[-1])

In [6]:
if os.path.isdir("/content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/taming-transformers/"):
  %cd "/content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/taming-transformers/"
else:
  %cd "/content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/"
  !git clone https://github.com/CompVis/taming-transformers
  %cd "taming-transformers"

/content/drive/MyDrive/4_ING_INFORMATICA/tfg/tfg/experiments/taming_transformers/taming-transformers


Cambiar la configuración del archivo .yaml para que apunte a los archivos de train y test correspondientes

### Entrenar el modelo con las imágenes de compuestos organometálicos

In [8]:
!python main.py --max_epochs 32 --base configs/custom_vqgan.yaml -t True --gpus 0,

Running on GPUs 0,
Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: ModelCheckpoint(save_last=True, monitor=None) is a redundant configuration. You can save the last checkpoint with ModelCheckpoint(save_top_k=None, monitor=None).
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
accumulate_grad_batches = 1
Setting learning rate to 2.25e-05 = 1 (accumulate_grad_batches) * 1 (num_gpus) * 5 (batchs